In [1]:
#!/usr/bin/env python

import argparse
import os
import sys
import numpy as np

import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from tensorboard_logger import configure, log_value

from models import Generator, Discriminator, FeatureExtractor
#from utils import Visualizer
import torchvision.utils as vutils
from PIL import Image,ImageFilter
from scipy.misc import imsave

imageSize = 32
upSampling = 2

transform = transforms.Compose([#transforms.RandomCrop(opt.imageSize*opt.upSampling),
                                 transforms.Scale(imageSize*upSampling),
                                transforms.ToTensor()])

normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                std = [0.229, 0.224, 0.225])

scale = transforms.Compose([transforms.ToPILImage(),
                            transforms.Scale(imageSize),
                            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                            transforms.ToTensor(),
                            transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                                std = [0.229, 0.224, 0.225])
                            ])

generator = Generator(16, upSampling)
generator.load_state_dict(torch.load('checkpoints/generator_final.pth'))


/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/shared/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


In [11]:
img = Image.open("../SN-GAN/log/fake_samples_epoch_005.png")
for i in range(8):
    for j in range(8):
        low_in = img.crop((34*i+2, 34*j+2,34*(i+1), 34*(j+1)))
        low_in.save('%s/low_res_%s.jpg' % ("output/low_res", str(i)+str(j)))

In [2]:
dataset = datasets.ImageFolder(root="output", transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16,
                                         shuffle=True, num_workers=2)

In [4]:
feature_extractor = FeatureExtractor(torchvision.models.vgg19(pretrained=True))
real_features = Variable(feature_extractor(high_res_real).data)
fake_features = feature_extractor(high_res_fake)
low_res_scale = torch.FloatTensor(16, 3, imageSize, imageSize)
for i,data in enumerate(dataloader):
    low_res, _ = data
    for j in range(16):
        low_res_scale[j] = scale(low_res[j])
    high_res_fake = generator(Variable(low_res_scale))
    vutils.save_image(high_res_fake.cpu().data,
                    'output/high_res_fake/fake_samples_epoch.jpg' )

NameError: name 'high_res_real' is not defined